In [1]:
import json

def load_batch(K=50, data_path='maps.json'):
    data = json.load(open(data_path))
    if K > len(data):
        raise IndexError("K should be no greater than size of dataset. size=" + str(len(data)))
    filenames = list(map(lambda x: x[0], data[:K]))
    objects = list(map(lambda x: x[1], data[:K]))
    feats = list(map(lambda x: x[2], data[:K]))
    return filenames, objects, feats

In [2]:
raw_names, raw_Y, raw_X = load_batch(2100)

In [3]:
X = raw_X

music_len = max(len(X[i]) for i in range(len(X)))
X_len = 65
Y_len = 4
num_hidden = 200

Y = [[[0, 0, 0, 0] for j in range(music_len)] for i in range(len(X))]
PX = [[0.5] * music_len for i in range(len(X))]
PY = [[0.5] * music_len for i in range(len(X))]
for i in range(len(X)):
    for j in range(music_len):
        if j < len(raw_Y[i]):
            Y[i][j][int(raw_Y[i][j][2])] = 1
            PX[i][j] = min(1, max(0, raw_Y[i][j][0]))
            PY[i][j] = min(1, max(0, raw_Y[i][j][1]))
        else:
            X[i].append([0]*X_len)

In [4]:
cnt = [0, 0, 0, 0]
for i in range(len(Y)):
    for j in range(len(Y[0])):
        for k in range(len(Y[0][0])):
            if Y[i][j][k] == 1:
                cnt[k] += 1
print cnt
print cnt[0] / 1.0 / sum(cnt), cnt[1] / 1.0 / sum(cnt), cnt[2] / 1.0 / sum(cnt), cnt[3] / 1.0 / sum(cnt)

[1370772, 412779, 1396248, 72735]
0.421447400704 0.126909972348 0.429280062868 0.0223625640808


In [4]:
num_train = 2000
t_names, t_X, t_Y, t_PX, t_PY = raw_names[num_train:], X[num_train:], Y[num_train:], PX[num_train:], PY[num_train:]
names, X, Y, PX, PY = raw_names[:num_train], X[:num_train], Y[:num_train], PX[:num_train], PY[:num_train]

In [5]:
import tensorflow as tf
data = tf.placeholder(tf.float32, [None, music_len, X_len])
target = tf.placeholder(tf.float32, [None, music_len, Y_len])
target_PX = tf.placeholder(tf.float32, [None, music_len])
target_PY = tf.placeholder(tf.float32, [None, music_len])

cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
output, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
# output: [batch, length, feature]

In [6]:
weight = tf.Variable(tf.truncated_normal([num_hidden, Y_len]))
bias = tf.Variable(tf.constant(0.0, shape=[Y_len]))

weight_PX = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias_PX = tf.Variable(tf.constant(0.5, shape=[1]))

weight_PY = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias_PY = tf.Variable(tf.constant(0.5, shape=[1]))

In [7]:
val = tf.reshape(output, [-1, num_hidden])
tar = tf.reshape(target, [-1, Y_len])
tar_PX = tf.reshape(target_PX, [-1])
tar_PY = tf.reshape(target_PY, [-1])

preds = tf.matmul(val, weight) + bias
preds_PX = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PX) + bias_PX, [-1]))
preds_PY = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PY) + bias_PY, [-1]))

In [8]:
cross_ens = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(preds, tar))
l2_PX = tf.reduce_mean((preds_PX - tar_PX) ** 2)
l2_PY = tf.reduce_mean((preds_PY - tar_PY) ** 2)

losses = cross_ens + l2_PX + l2_PY

In [9]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(losses)

In [10]:
mistakes = tf.not_equal(tf.argmax(tar, 1), tf.argmax(preds, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [16]:
res, res_PX, res_PY = sess.run((preds, preds_PX, preds_PY), {data:t_X})

In [19]:
print type(res)

<type 'numpy.ndarray'>


In [20]:
with open('filenames.json', 'w') as f:
    json.dump(t_names, f)
res.dump('labels')
res_PX.dump('PX')
res_PY.dump('PY')

In [12]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [13]:
batch_size = 100
no_of_batches = int(len(X) / batch_size)
dfm = {data:t_X, target:t_Y, target_PX:t_PX, target_PY:t_PY}
print len(X), no_of_batches

2000 20


In [15]:
for i in range(50):
    ptr = 0
    for j in range(no_of_batches):
        inp, out, out_PX, out_PY = X[ptr:ptr+batch_size], Y[ptr:ptr+batch_size], PX[ptr:ptr+batch_size], PY[ptr:ptr+batch_size]
        ptr += batch_size
        ass = {data: inp, target: out, target_PX: out_PX, target_PY: out_PY}
        sess.run(minimize, ass)
#         incor = sess.run(error, ass)
#         loss = sess.run(losses, ass)
#         cross = sess.run(cross_ens, ass)
#         lx = sess.run(l2_PX, ass)
#         ly = sess.run(l2_PY, ass)
#         print('Epoch {:2d} loss {:3.6f}'.format(i + 1, loss))
#         print('         error {:3.6f}% cross {:3.6f} lx {:3.6f} ly {:3.6f}'.format(incor*100, cross, lx, ly))
#         print i, j
    incor, loss, cross, lx, ly = sess.run((error, losses, cross_ens, l2_PX, l2_PY), dfm)
    print('Epoch {:2d} loss {:3.6f}'.format(i + 1, loss))
    print('         error {:3.6f}% cross {:3.6f} lx {:3.6f} ly {:3.6f}'.format(incor*100, cross, lx, ly))

Epoch  1 loss 1.488993
         error 66.329676% cross 1.269516 lx 0.120590 ly 0.098887
Epoch  2 loss 1.320009
         error 65.599805% cross 1.162623 lx 0.077307 ly 0.080079
Epoch  3 loss 1.232096
         error 65.304464% cross 1.098337 lx 0.065489 ly 0.068270
Epoch  4 loss 1.174245
         error 65.021592% cross 1.056257 lx 0.058099 ly 0.059890
Epoch  5 loss 1.136834
         error 64.854449% cross 1.028114 lx 0.053604 ly 0.055116
Epoch  6 loss 1.109133
         error 64.758587% cross 1.008532 lx 0.049657 ly 0.050944
Epoch  7 loss 1.088997
         error 64.593774% cross 0.994406 lx 0.046658 ly 0.047933
Epoch  8 loss 1.074741
         error 64.475316% cross 0.984651 lx 0.044537 ly 0.045553
Epoch  9 loss 1.062824
         error 64.365858% cross 0.976058 lx 0.042903 ly 0.043863
Epoch 10 loss 1.054734
         error 64.179927% cross 0.970225 lx 0.041745 ly 0.042764
Epoch 11 loss 1.047977
         error 64.120495% cross 0.965326 lx 0.040907 ly 0.041744
Epoch 12 loss 1.044149
         

In [21]:
sess.close()